# Analyse de Data Drift avec Evidently

Dans le cadre de la Mission 2 du projet de scoring crédit "Prêt à dépenser", nous implémentons une stratégie de suivi de la performance du modèle en production. Cette étape cruciale du cycle MLOps permet de détecter d'éventuelles dégradations des données d'entrée susceptibles d'affecter la fiabilité des prédictions.
<br>
Le data drift représente l'évolution des distributions statistiques des variables d'entrée entre la phase d'entraînement et la production. 
Ce phénomène peut compromettre les performances du modèle et nécessiter un réentraînement.
<br>
Les causes principales du drift peuvent être :
- Évolution du marché : Inflation, nouveaux produits financiers, changements réglementaires
- Saisonnalité : Variations cycliques des demandes de crédit
- Changements comportementaux : Évolution des habitudes des clients
- Biais de collecte : Modifications dans les processus de saisie des données
<br>

Une solution Technique : Evidently<br>
Evidently est une librairie Python spécialisée dans le monitoring des modèles de machine learning. Elle permet de :
- Détecter automatiquement le data drift sur l'ensemble des variables
- Appliquer des tests statistiques adaptés (Kolmogorov-Smirnov, Jensen-Shannon, etc.)
- Générer des rapports HTML interactifs avec visualisations
- Intégrer facilement avec MLflow pour le tracking
<br>

Conformément au cahier des charges, nous analysons le data drift en comparant :
- Dataset de référence : application_train.csv (données d'entraînement du modèle)
- Dataset courant : application_test.csv (simulation de nouveaux clients en production)
<br>

L'analyse porte sur les 121 variables communes entre les deux jeux de données, permettant une détection précoce des dérives avant qu'elles n'impactent les performances métier.
<br>

Coté livrables nous optenons un rapport HTML (Tableau d'analyse détaillé avec visualisations par variable) et les métriques MLflow (Suivi  des indicateurs de drift)

In [1]:
"""
Comparer application_train vs application_test
"""

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Evidently
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# MLflow
import mlflow
import mlflow.sklearn

# Configuration MLflow
mlflow.set_tracking_uri("http://localhost:5000")
experiment_name = "4-credit-scoring-data-drift"
mlflow.set_experiment(experiment_name)

# Configuration des chemins
project_dir = Path().absolute()
if project_dir.name in ['reports', 'notebooks']:
    project_dir = project_dir.parent

data_dir = project_dir / 'data' / 'source'
reports_dir = project_dir / 'reports'
reports_dir.mkdir(exist_ok=True)

# Vérification des fichiers
train_file = data_dir / 'application_train.csv'
test_file = data_dir / 'application_test.csv'

print(f"Répertoire projet: {project_dir}")
print(f"Répertoire données: {data_dir}")
print(f"Répertoire rapports: {reports_dir}")

if not train_file.exists() or not test_file.exists():
    raise FileNotFoundError(f"Fichiers manquants:\n- {train_file}\n- {test_file}")

print(f" Fichiers trouvés:")
print(f"   - Train: {train_file}")
print(f"   - Test:  {test_file}")

# Démarrer MLflow run
mlflow_run = mlflow.start_run(run_name="evidently_data_drift_analysis")
print(f" MLflow run démarré: {mlflow_run.info.run_id}")

print("\n Chargement des données...")

# Chargement des données selon le cahier des charges
print("   - Chargement application_train.csv (référence)...")
reference_data = pd.read_csv(train_file)
print(f"     Shape: {reference_data.shape}")

print("   - Chargement application_test.csv (production)...")
current_data = pd.read_csv(test_file)
print(f"     Shape: {current_data.shape}")

# Alignment des colonnes (test n'a pas de TARGET)
common_columns = [col for col in current_data.columns if col in reference_data.columns]
ref_aligned = reference_data[common_columns].copy()
curr_aligned = current_data[common_columns].copy()

print(f"\n Alignement des datasets:")
print(f"   - Colonnes communes: {len(common_columns)}")
print(f"   - Reference aligned: {ref_aligned.shape}")
print(f"   - Current aligned: {curr_aligned.shape}")

# Variables exclues
excluded_vars = set(reference_data.columns) - set(common_columns)
print(f"   - Variables exclues: {len(excluded_vars)} (TARGET principalement)")

# Tracking MLflow des paramètres
mlflow.log_param("reference_dataset", "application_train.csv")
mlflow.log_param("current_dataset", "application_test.csv")
mlflow.log_param("reference_samples", reference_data.shape[0])
mlflow.log_param("current_samples", current_data.shape[0])
mlflow.log_param("reference_features", reference_data.shape[1])
mlflow.log_param("current_features", current_data.shape[1])
mlflow.log_param("analyzed_features", len(common_columns))
mlflow.log_param("excluded_features", len(excluded_vars))
mlflow.log_param("analysis_date", datetime.now().isoformat())
mlflow.log_param("full_dataset_analysis", True)
mlflow.log_param("analyst", "Brice_Béchet")

print("\n Création du rapport Evidently...")

# Création et exécution du rapport Evidently
report = Report(metrics=[DataDriftPreset()])

print("   - Analyse en cours (peut prendre quelques minutes)...")
report.run(reference_data=ref_aligned, current_data=curr_aligned)

print("    Analyse terminée")

# Sauvegarde du rapport HTML
report_path = reports_dir / 'data_drift_evidently_report.html'
report.save_html(str(report_path))

print(f"\n Rapport HTML sauvegardé: {report_path}")

# Extraction des métriques du rapport
print("\n Extraction des métriques...")
report_dict = report.as_dict()

try:
    metric_result = report_dict['metrics'][0]['result']
    
    # Métriques principales
    dataset_drift = metric_result.get('dataset_drift', False)
    drift_share = metric_result.get('drift_share', 0.0)
    n_drifted_columns = metric_result.get('number_of_drifted_columns', 0)
    
    # Informations détaillées sur les colonnes avec drift
    drifted_features = []
    if 'drift_by_columns' in metric_result:
        drift_by_columns = metric_result['drift_by_columns']
        for col, drift_info in drift_by_columns.items():
            if drift_info.get('drift_detected', False):
                p_value = drift_info.get('stattest_result', {}).get('pvalue', 'N/A')
                drifted_features.append((col, p_value))
    
    # Logging MLflow des métriques
    mlflow.log_metric("dataset_drift_detected", 1 if dataset_drift else 0)
    mlflow.log_metric("drift_share", drift_share)
    mlflow.log_metric("drifted_features_count", n_drifted_columns)
    mlflow.log_metric("total_features_analyzed", len(common_columns))
    mlflow.log_metric("drift_percentage", (n_drifted_columns / len(common_columns)) * 100)
    
    # Logging des features avec drift
    if drifted_features:
        mlflow.log_param("drifted_features_list", [feat[0] for feat in drifted_features[:10]])  # Top 10
    
    # Résultats
    print("Résultat de l'analyse")
    print(f" Dataset drift détecté: {'OUI' if dataset_drift else 'NON'}")
    print(f" Proportion de drift: {drift_share:.1%}")
    print(f" Features avec drift: {n_drifted_columns}/{len(common_columns)} ({(n_drifted_columns/len(common_columns)*100):.1f}%)")
    
    if drifted_features:
        print(f"\n Top features avec drift détecté:")
        for i, (feature, p_value) in enumerate(drifted_features[:10], 1):
            print(f"   {i:2d}. {feature} (p-value: {p_value})")
    
    # Interprétation métier
    print(f"\n INTERPRÉTATION:")
    if dataset_drift:
        print("  Data drift détecté!")
    else:
        print(" Aucun drift significatif détecté")

    # Statut de la mission
    mlflow.log_param("mission_status", "completed_successfully")
    mlflow.log_param("drift_detected", dataset_drift)
    mlflow.log_param("interpretation", "drift_detected" if dataset_drift else "no_significant_drift")
    
except Exception as e:
    print(f" Erreur lors de l'extraction des métriques: {e}")
    mlflow.log_param("mission_status", "completed_with_extraction_error")
    mlflow.log_param("error_details", str(e))

# Artifacts
mlflow.log_artifact(str(report_path))

print(f"\n Rapport HTML: {report_path}")
print(f" MLflow UI: http://localhost:5000")
print(f" Run ID: {mlflow_run.info.run_id}")


# Fermeture du run MLflow
mlflow.end_run()

Répertoire projet: C:\Users\beche\Documents\Formation\Projet7\credit-scoring-project
Répertoire données: C:\Users\beche\Documents\Formation\Projet7\credit-scoring-project\data\source
Répertoire rapports: C:\Users\beche\Documents\Formation\Projet7\credit-scoring-project\reports
 Fichiers trouvés:
   - Train: C:\Users\beche\Documents\Formation\Projet7\credit-scoring-project\data\source\application_train.csv
   - Test:  C:\Users\beche\Documents\Formation\Projet7\credit-scoring-project\data\source\application_test.csv
 MLflow run démarré: a7a51cbe0b3642a7a472cfb6c3d5cdeb

 Chargement des données...
   - Chargement application_train.csv (référence)...
     Shape: (307511, 122)
   - Chargement application_test.csv (production)...
     Shape: (48744, 121)

 Alignement des datasets:
   - Colonnes communes: 121
   - Reference aligned: (307511, 121)
   - Current aligned: (48744, 121)
   - Variables exclues: 1 (TARGET principalement)

 Création du rapport Evidently...
   - Analyse en cours (peut 

### Variables en Drift Identifiées

Les 9 variables présentant un drift statistiquement significatif :

| **Variable** | **Type** | **Drift Score** | **Test Statistique** | **Niveau de Drift** |
|--------------|----------|-----------------|---------------------|---------------------|
| `AMT_REQ_CREDIT_BUREAU_QRT` | Numérique | 0.359 | Wasserstein | Fort |
| `AMT_REQ_CREDIT_BUREAU_MON` | Numérique | 0.282 | Wasserstein | Modéré |
| `AMT_GOODS_PRICE` | Numérique | 0.211 | Wasserstein | Modéré |
| `AMT_CREDIT` | Numérique | 0.207 | Wasserstein | Modéré |
| `AMT_ANNUITY` | Numérique | 0.161 | Wasserstein | Modéré |
| `AMT_REQ_CREDIT_BUREAU_WEEK` | Numérique | 0.154 | Wasserstein | Modéré
| `NAME_CONTRACT_TYPE` | Catégorielle | 0.148 | Jensen-Shannon | Modéré |
| `DAYS_LAST_PHONE_CHANGE` | Numérique | 0.139 | Wasserstein | Modéré |
| `FLAG_EMAIL` | Binaire | 0.122 | Jensen-Shannon | Faible |


- 94% des variables (112/121) demeurent stables
- Les scores externes (EXT_SOURCE) et principales variables démographiques ne présentent pas de drift

Les variables de montants (`AMT_CREDIT`, `AMT_ANNUITY`, `AMT_GOODS_PRICE`) montrent des variations qui peuvent s'expliquer par l'évolution des taux d'emprunt sur le marché bancaire, l'évolution de la clientèle ... (nouveaux segments ?) 

Les varaibles AMT_REQ_CREDIT_BUREAU_QRT, AMT_REQ_CREDIT_BUREAU_MON et AMT_REQ_CREDIT_BUREAU_WEEK sont relatives au dossier de demandes de crédits par les clients. Plus precisement au nombre de fois ou les dossiers ont été consultés. La hausse des requêtes au bureau de crédit peut etre en lien avec des habitudes de clients mettant en concurrence les banques par exemple.

Les changements dans `DAYS_LAST_PHONE_CHANGE` et `FLAG_EMAIL` concernent l'évoltution naturelle des moyens de communication et des habitudes des clients.

Le modèle de scoring demeure fiable et opérationnel (seulement 7.4% de variables présentant un drift)